<a href="https://colab.research.google.com/github/gcasaldi/CS50/blob/main/Un_benvenuto_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl
import pandas as pd
from datetime import datetime, timedelta
from google.colab import files
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from collections import Counter





Caricare file .txt e visualizzare il file

Creare il DataFrame con i numeri estratti





In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

In [3]:
from google.colab import files
import zipfile
import os

# Carica il file .zip
uploaded = files.upload()

# Estrazione del contenuto del file .zip
for fn in uploaded.keys():
    if fn.endswith(".zip"):
        # Creiamo una cartella per estrarre i file
        extract_folder = '/content/extracted_files'
        os.makedirs(extract_folder, exist_ok=True)

        # Estraiamo i contenuti del .zip
        with zipfile.ZipFile(fn, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
            print(f"File estratti in: {extract_folder}")

# Lista i file estratti per assicurarti che ci sia un .txt
extracted_files = os.listdir(extract_folder)
print("File estratti:", extracted_files)

# Se c'è un file .txt, lo leggiamo
for file in extracted_files:
    if file.endswith(".txt"):
        txt_file_path = os.path.join(extract_folder, file)
        with open(txt_file_path, 'r') as f:
            data = f.readlines()

        print(f"Contenuto del file {file}:\n")
        print(data[:10])  # Visualizza le prime 5 righe per verificarne il contenuto


Saving SuperEnalotto-archivio-estrazioni-2025.zip to SuperEnalotto-archivio-estrazioni-2025.zip
File estratti in: /content/extracted_files
File estratti: ['SuperEnalotto-archivio-estrazioni-2025.txt']
Contenuto del file SuperEnalotto-archivio-estrazioni-2025.txt:

['SuperEnalotto\n', 'Archivio estrazioni SuperEnalotto anno 2025 aggiornato al 11/03/2025\n', 'Concorso\tData\tN.1\tN.2\tN.3\tN.4\tN.5\tN.6\tJolly\tSuperStar\t\n', '40\t11/03/2025\t22\t27\t48\t55\t58\t90\t63\t11\t\n', '39\t08/03/2025\t11\t16\t35\t59\t65\t87\t9\t52\t\n', '38\t07/03/2025\t11\t24\t25\t30\t43\t47\t28\t5\t\n', '37\t06/03/2025\t1\t10\t18\t19\t32\t36\t20\t53\t\n', '36\t04/03/2025\t9\t28\t52\t63\t77\t79\t11\t39\t\n', '35\t01/03/2025\t1\t12\t33\t40\t41\t83\t6\t67\t\n', '34\t28/02/2025\t4\t12\t37\t59\t63\t90\t70\t15\t\n']


ripulire


In [4]:
import pandas as pd

# Percorso del file .txt estratto
txt_file_path = '/content/extracted_files/SuperEnalotto-archivio-estrazioni-2025.txt'

# Leggi il file
with open(txt_file_path, 'r') as f:
    data = f.readlines()

# Pulizia e parsing dei dati
data_cleaned = [line.strip().split('\t') for line in data]

# Trova l'indice della riga di intestazione (dovrebbe essere la terza riga)
header_index = next(i for i, row in enumerate(data_cleaned) if "Concorso" in row)

# Estrai i dati a partire dall'intestazione
columns = data_cleaned[header_index]
rows = data_cleaned[header_index + 1:]

# Creazione del DataFrame
df = pd.DataFrame(rows, columns=columns)

# Converte i numeri da stringhe a interi
df.iloc[:, 2:] = df.iloc[:, 2:].apply(pd.to_numeric, errors='coerce')

# Visualizza le prime estrazioni per controllo
print("Prime 10 estrazioni del SuperEnalotto:")
print(df.head(10))


Prime 10 estrazioni del SuperEnalotto:
  Concorso        Data N.1 N.2 N.3 N.4 N.5 N.6 Jolly SuperStar
0       40  11/03/2025  22  27  48  55  58  90    63        11
1       39  08/03/2025  11  16  35  59  65  87     9        52
2       38  07/03/2025  11  24  25  30  43  47    28         5
3       37  06/03/2025   1  10  18  19  32  36    20        53
4       36  04/03/2025   9  28  52  63  77  79    11        39
5       35  01/03/2025   1  12  33  40  41  83     6        67
6       34  28/02/2025   4  12  37  59  63  90    70        15
7       33  27/02/2025  29  31  33  42  51  72    34        62
8       32  25/02/2025   6  58  68  83  89  90    73        50
9       31  22/02/2025  35  40  48  49  52  89    24        32


Creazione del Dataset

In [47]:
import pandas as pd

# Supponiamo che 'df' sia il DataFrame già creato
# Creazione del Dataset
data = []
for i in range(len(df) - 1):
    current_row = df.iloc[i]
    next_row = df.iloc[i + 1]

    # Correggi la selezione delle colonne:
    # Seleziona tutte le colonne numeriche (N1, N2, ..., Jolly, SuperStar)
    data.append(current_row[['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'Jolly', 'SuperStar']].tolist() +
                next_row[['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'Jolly', 'SuperStar']].tolist())

# Creiamo un nuovo DataFrame
columns = ['N.1', 'N.2', 'N.3', 'N.4', 'N.5', 'N.6', 'Jolly', 'SuperStar'] + ['Next_N.1', 'Next_N.2', 'Next_N.3', 'Next_N.4', 'Next_N.5', 'Next_N.6', 'Next_Jolly', 'Next_SuperStar']
training_df = pd.DataFrame(data, columns=columns)

# Visualizziamo il DataFrame di training
print(training_df.head())

   N.1  N.2  N.3  N.4  N.5  N.6  Jolly  SuperStar  Next_N.1  Next_N.2  \
0   22   27   48   55   58   90     63         11        11        16   
1   11   16   35   59   65   87      9         52        11        24   
2   11   24   25   30   43   47     28          5         1        10   
3    1   10   18   19   32   36     20         53         9        28   
4    9   28   52   63   77   79     11         39         1        12   

   Next_N.3  Next_N.4  Next_N.5  Next_N.6  Next_Jolly  Next_SuperStar  
0        35        59        65        87           9              52  
1        25        30        43        47          28               5  
2        18        19        32        36          20              53  
3        52        63        77        79          11              39  
4        33        40        41        83           6              67  


Addestramento del Modello

In [52]:
import pandas as pd

# Supponiamo che 'df' sia il DataFrame già creato
# Creazione del Dataset
data = []
for i in range(len(df) - 1):
    current_row = df.iloc[i]
    next_row = df.iloc[i + 1]

    # Correggi la selezione delle colonne:
    # Seleziona tutte le colonne numeriche (N1, N2, ..., Jolly, SuperStar)
    data.append(current_row[['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'Jolly', 'SuperStar']].tolist() +
                next_row[['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'Jolly', 'SuperStar']].tolist())

# Creiamo un nuovo DataFrame
columns = ['N.1', 'N.2', 'N.3', 'N.4', 'N.5', 'N.6', 'Jolly', 'SuperStar'] + ['Next_N.1', 'Next_N.2', 'Next_N.3', 'Next_N.4', 'Next_N.5', 'Next_N.6', 'Next_Jolly', 'Next_SuperStar']
training_df = pd.DataFrame(data, columns=columns)

# Visualizziamo il DataFrame di training
print(training_df.head())

   N.1  N.2  N.3  N.4  N.5  N.6  Jolly  SuperStar  Next_N.1  Next_N.2  \
0   22   27   48   55   58   90     63         11        11        16   
1   11   16   35   59   65   87      9         52        11        24   
2   11   24   25   30   43   47     28          5         1        10   
3    1   10   18   19   32   36     20         53         9        28   
4    9   28   52   63   77   79     11         39         1        12   

   Next_N.3  Next_N.4  Next_N.5  Next_N.6  Next_Jolly  Next_SuperStar  
0        35        59        65        87           9              52  
1        25        30        43        47          28               5  
2        18        19        32        36          20              53  
3        52        63        77        79          11              39  
4        33        40        41        83           6              67  


Previsione della Prossima Estrazione

In [54]:
# prompt: allena il modello e fai la previsione

# Preparazione dei dati per il modello
X = training_df.drop(columns=['Next_N.1', 'Next_N.2', 'Next_N.3', 'Next_N.4', 'Next_N.5', 'Next_N.6', 'Next_Jolly', 'Next_SuperStar'])
y = training_df[['Next_N.1', 'Next_N.2', 'Next_N.3', 'Next_N.4', 'Next_N.5', 'Next_N.6', 'Next_Jolly', 'Next_SuperStar']]

# Dividi i dati in set di training e testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizzazione dei dati
scaler_x = StandardScaler()
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)

scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# Crea e addestra il modello (esempio con una rete neurale)
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1]))  # Output layer con 8 neuroni
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1) # Riduce le epoche per test rapidi

# Previsione sulla base dell'ultima riga del DataFrame originale
last_row = df.iloc[-1][['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'Jolly', 'SuperStar']].values.reshape(1,-1)
last_row_scaled = scaler_x.transform(last_row)
prediction_scaled = model.predict(last_row_scaled)

# Inverto la standardizzazione per ottenere i numeri originali
prediction = scaler_y.inverse_transform(prediction_scaled)

# Arrotonda i valori previsti (i numeri del lotto sono interi)
prediction_rounded = np.round(prediction).astype(int)


print("Prossima estrazione prevista:")
prediction_rounded


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1.1284
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 1.0905
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 1.0577
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 1.0292
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 1.0047
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9836
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.9651
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.9486
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.9335
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.9197
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.9067
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.8945
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.8829
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.8716
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.8606
Epoch 16/100
1/1 ━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[ 8, 21, 16, 51, 59, 65, 65, 33]])

In [58]:
# prompt: allena di nuovo il modello con più sicurezza sulla base statistica dei numeri usciti. tieni da conto quelli che escono di più e quelli che sono in ritardo. tieni da conto le estrazioni storiche già uscite e fai un training sofisticato che sia preciso sulla prossima estrazione

# ... (Your existing code)

# Feature Engineering: Include historical frequency and recency of numbers
from collections import Counter # Import Counter from collections module

def calculate_frequencies(df, column_name, window_size=50): # Add window size parameter
    frequencies = []
    for i in range(len(df)):
        # Calculate frequencies within the sliding window
        window = df[column_name][max(0, i - window_size + 1):i + 1]
        counts = Counter(window)
        frequencies.append({num: count for num, count in counts.items()})
    return frequencies


# Apply to all relevant columns (N1-N6, Jolly, SuperStar)
for col in ['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'Jolly', 'SuperStar']:
    df[f'{col}_freq'] = calculate_frequencies(df, col)


# ... (rest of your data preprocessing code)

# Model Training (using a more complex model and more data)

# ... (your existing model code)

# Consider a deeper neural network or other suitable models like LSTM or Random Forest

model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))  # Increased neurons
model.add(Dropout(0.2))  # Add dropout for regularization
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1]))
model.compile(optimizer='adam', loss='mse')

# Increase the number of epochs and adjust the batch size as needed
model.fit(X_train, y_train, epochs=200, batch_size=64, verbose=1) # Increased Epochs and batch size

# ... (rest of your prediction code)


# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss}")

print("Prossima estrazione prevista:")
prediction_rounded


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1.0156
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 1.0208
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.9706
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.9832
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.9680
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.9448
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.9341
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.9280
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.9280
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.9082
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.8983
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.8718
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.8569
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.8467
Epoch 15/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.8583
Epoch 16/200
1/1 ━━━━━━━━━━━━━━━━

array([[ 8, 20, 25, 39, 56, 65, 47, 36]])

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Carica il file
txt_file_path = '/content/extracted_files/SuperEnalotto-archivio-estrazioni-2025.txt'

with open(txt_file_path, 'r') as f:
    data = f.readlines()

# Step 1: Pulizia dati ed estrazione frequenze e ritardi
data_cleaned = data[3:]  # Ignora intestazioni
extraction_data = []

for line in data_cleaned:
    elements = line.strip().split('\t')
    if len(elements) == 10:  # Verifica che ci siano abbastanza colonne
        date, num1, num2, num3, num4, num5, num6, jolly, superstar = elements[1:]
        extraction_data.append([int(num1), int(num2), int(num3), int(num4), int(num5), int(num6), int(jolly), int(superstar)])

df = pd.DataFrame(extraction_data, columns=["N1", "N2", "N3", "N4", "N5", "N6", "Jolly", "SuperStar"])

# Calcolo frequenze di ogni numero
all_numbers = df.iloc[:, :6].values.flatten()
number_counts = pd.Series(all_numbers).value_counts().sort_index()

# Calcolo ritardi (da quante estrazioni manca un numero)
latest_occurrence = {}
for num in range(1, 91):
    occurrences = df[df.iloc[:, :6].eq(num).any(axis=1)].index
    latest_occurrence[num] = occurrences[-1] if len(occurrences) > 0 else -1

delays = {num: (len(df) - latest_occurrence[num] - 1 if latest_occurrence[num] != -1 else len(df)) for num in range(1, 91)}


# Step 2: Predire il range dei numeri più probabili
probability_scores = {num: (number_counts.get(num, 0) / len(df) * 100) for num in range(1, 91)}
sorted_numbers = sorted(probability_scores.keys(), key=lambda x: (probability_scores[x], -delays[x]), reverse=True)
predicted_range = sorted_numbers[:20]  # Prendiamo i 20 numeri più probabili

print(f"🔍 Numeri più probabili basati su frequenza e ritardo: {predicted_range}")







🔍 Numeri più probabili basati su frequenza e ritardo: [72, 29, 77, 40, 41, 30, 63, 83, 11, 85, 48, 19, 21, 74, 49, 89, 31, 86, 7, 10]


Analisi

In [59]:
# Esamina i numeri in predicted_range
for num in predicted_range:
    freq = number_counts.get(num, 0)  # Frequenza del numero
    delay = delays.get(num, 0)  # Ritardo del numero
    print(f"Numero: {num}, Frequenza: {freq}, Ritardo: {delay}")


Numero: 72, Frequenza: 9, Ritardo: 5
Numero: 29, Frequenza: 7, Ritardo: 0
Numero: 77, Frequenza: 6, Ritardo: 0
Numero: 40, Frequenza: 6, Ritardo: 4
Numero: 41, Frequenza: 6, Ritardo: 11
Numero: 30, Frequenza: 5, Ritardo: 1
Numero: 63, Frequenza: 5, Ritardo: 2
Numero: 83, Frequenza: 5, Ritardo: 2
Numero: 11, Frequenza: 5, Ritardo: 3
Numero: 85, Frequenza: 5, Ritardo: 6
Numero: 48, Frequenza: 5, Ritardo: 15
Numero: 19, Frequenza: 4, Ritardo: 0
Numero: 21, Frequenza: 4, Ritardo: 0
Numero: 74, Frequenza: 4, Ritardo: 0
Numero: 49, Frequenza: 4, Ritardo: 1
Numero: 89, Frequenza: 4, Ritardo: 1
Numero: 31, Frequenza: 4, Ritardo: 2
Numero: 86, Frequenza: 4, Ritardo: 2
Numero: 7, Frequenza: 4, Ritardo: 3
Numero: 10, Frequenza: 4, Ritardo: 3


ALLENAMENTO

In [60]:
# X contiene i numeri estratti come input (6 numeri)
X = df.iloc[:-1, :6].values  # Numeri estratti (eliminando l'ultima riga)

# y contiene i 6 numeri + Jolly + SuperStar come output (dalla seconda riga)
y = df.iloc[1:, :8].values   # 6 numeri + Jolly + SuperStar (eliminando la prima riga)

# Controlla le forme per essere sicuro che X e y abbiano la stessa lunghezza
print("Forma di X:", X.shape)
print("Forma di y:", y.shape)

# Normalizzazione delle feature
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)  # Normalizzazione separata per y

# Modello di rete neurale
model = Sequential([
    Dense(128, input_dim=6, activation='relu'),
    Dense(64, activation='relu'),
    Dense(8, activation='linear')  # 6 numeri + Jolly + SuperStar
])

model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

# Training
model.fit(X_scaled, y_scaled, epochs=200, batch_size=10, verbose=1)
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Forma di X: (39, 6)
Forma di y: (39, 8)
Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.0765 - mae: 0.8598  
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9457 - mae: 0.8095 
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.0157 - mae: 0.8362 
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.8777 - mae: 0.7917 
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.8627 - mae: 0.7752 
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.8791 - mae: 0.7882 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8795 - mae: 0.7858 
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.8855 - mae: 0.7859
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9186 - mae: 0.8122
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7973 - mae: 0.7612 
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9001 - mae: 0.7856 
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.8567 - mae: 0.7647 
Epoch 13/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.

PREVISIONI

In [61]:
# Previsione sulla prossima estrazione
# L'ultima estrazione di numeri (X) che utilizzeremo come input
last_entry = df.iloc[-1, :6].values.reshape(1, -1)  # Reshape per essere compatibile con il modello

# Normalizzazione dell'ultima estrazione (usando lo scaler già creato)
last_entry_scaled = scaler_X.transform(last_entry)

# Previsione con il modello
prediction_scaled = model.predict(last_entry_scaled)

# Invertiamo la normalizzazione per ottenere i numeri reali
prediction = scaler_y.inverse_transform(prediction_scaled)

# Estraiamo i numeri previsti
predicted_numbers = np.round(prediction[0][:6]).astype(int)  # I 6 numeri principali
predicted_jolly = int(np.round(prediction[0][6]))  # Jolly
predicted_superstar = int(np.round(prediction[0][7]))  # SuperStar

# Stampa la previsione
print("🔮 Previsione per la prossima estrazione:")
print(f"🎰 Numeri: {predicted_numbers}")
print(f"🃏 Jolly previsto: {predicted_jolly}")
print(f"🌟 SuperStar previsto: {predicted_superstar}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
🔮 Previsione per la prossima estrazione:
🎰 Numeri: [ 9 18 24 31 48 60]
🃏 Jolly previsto: 42
🌟 SuperStar previsto: 31


Normalizza i dati e fai previsioni

In [4]:
#nooo

from google.colab import files
import zipfile
import os

# Carica il file ZIP
uploaded = files.upload()

# Verifica il nome del file caricato
file_name = list(uploaded.keys())[0]
zip_path = f'/content/{file_name}'

# Estrai il contenuto del file ZIP
extracted_folder = '/content/extracted/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# Verifica i file estratti
extracted_files = os.listdir(extracted_folder)
print(f"File estratti: {extracted_files}")

# Salva il path del file Excel estratto per usarlo nella seconda cella
excel_file_path = os.path.join(extracted_folder, extracted_files[0])
print(f"File Excel estratto: {excel_file_path}")



KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Usa il path ottenuto dalla prima cella
file_path = '/content/extracted/it-superenalotto-past-draws-archive.xls'  # Assicurati che questo percorso sia corretto

def carica_database(file_path):
    # Usa 'xlrd' come engine per i file .xls
    df = pd.read_excel(file_path, engine='xlrd', skiprows=1)
    numeri_estratti = df.select_dtypes(include='number').dropna(axis=1, how='all')
    numeri_estratti = numeri_estratti.tail(500)
    return numeri_estratti

# Carica i dati nel DataFrame
df = carica_database(file_path)

# Visualizza le prime 10 righe
print(df.head(10))


Codice per ordinare e correggere i dati:

In [ ]:
# Pulizia dei dati: rimuoviamo le colonne vuote e manteniamo solo i numeri
def pulizia_dati(df):
    df_cleaned = df.select_dtypes(include='number').dropna(axis=1, how='all')  # Manteniamo solo colonne con numeri
    return df_cleaned

# Applichiamo la pulizia
df_cleaned = pulizia_dati(df)

# Mostra le prime 10 righe dopo la pulizia
print(df_cleaned.head(10))


   Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7
0         NaN         NaN         NaN         NaN         NaN
1         NaN         NaN         NaN         NaN         NaN
2         2.0         3.0         4.0         5.0         6.0
3        16.0        35.0        59.0        65.0        87.0
4        24.0        25.0        30.0        43.0        47.0
5        10.0        18.0        19.0        32.0        36.0
6        28.0        52.0        63.0        77.0        79.0
7        12.0        33.0        40.0        41.0        83.0
8        12.0        37.0        59.0        63.0        90.0
9        31.0        33.0        42.0        51.0        72.0


Seleziona solo le colonne numeriche (i numeri estratti)

In [ ]:
# Seleziona solo le colonne numeriche (i numeri estratti)
df_cleaned = df.select_dtypes(include=[np.number])

# Rimuove righe che contengono tutti valori NaN
df_cleaned = df_cleaned.dropna(how='all')

# Converte i numeri in interi per eliminare i .0
df_cleaned = df_cleaned.astype(int)

# Stampa i primi numeri per verificare
print(df_cleaned.head(10))

    Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7
2            2           3           4           5           6
3           16          35          59          65          87
4           24          25          30          43          47
5           10          18          19          32          36
6           28          52          63          77          79
7           12          33          40          41          83
8           12          37          59          63          90
9           31          33          42          51          72
10          58          68          83          89          90
11          40          48          49          52          89



# Prendi solo le ultime 500 estrazioni

In [ ]:
df_cleaned = df_cleaned.tail(800)

# Normalizza i dati tra 0 e 1

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df_cleaned)



# Funzione per creare sequenze di dati

In [ ]:
def prepara_dati(df_scaled, sequenza_lunghezza=5):
    X, y = [], []
    for i in range(len(df_scaled) - sequenza_lunghezza):
        X.append(df_scaled[i:i + sequenza_lunghezza])
        y.append(df_scaled[i + sequenza_lunghezza])  # Previsione successiva
    return np.array(X), np.array(y)


# Prepara i dati per il modello LSTM

In [ ]:
def prepara_dati(df_scaled, sequenza_lunghezza=5):
    X, y = [], []
    for i in range(len(df_scaled) - sequenza_lunghezza):  # Modifica qui
        X.append(df_scaled[i:i + sequenza_lunghezza])  # Aggiungi sequenza di lunghezza 5
        y.append(df_scaled[i + sequenza_lunghezza])  # Previsione successiva
    return np.array(X), np.array(y)


Creiamo il modello LSTM e lo alleniamo

In [ ]:
# Prepara i dati
X, y = prepara_dati(df_scaled)

# Dividi i dati in set di training e testing
split_index = int(len(X) * 0.8)  # Usa l'80% per il training
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Crea il modello LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1]))  # Output layer con numero di unità pari a features

# Compila il modello
model.compile(optimizer='adam', loss='mean_squared_error')

# Allena il modello
model.fit(X_train, y_train, epochs=100, batch_size=32)  #epochs=50, batch_size=32

# Valuta il modello
loss = model.evaluate(X_test, y_test)
print(f"Loss sul set di test: {loss}")

# Fai delle previsioni
predictions = model.predict(X_test)

# Inverti la normalizzazione per ottenere le previsioni nella scala originale
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# Stampa le prime 10 previsioni
print("Prime 10 previsioni:")
print(predictions[:10])

# Stampa le prime 10 osservazioni reali
print("\nPrime 10 osservazioni reali:")
print(y_test[:10])



Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.4807
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.4516
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.4218
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.3953
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.3680
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.3451
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.3193
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.2940
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2633
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.2390
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.2281
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1894
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.1677
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1382
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.1136
Epoch 16/100
1/1 ━━━━

Ottimizzazione del modello mix tra dati statistici e training

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Funzione per calcolare la similarità tra la previsione e l'estrazione
def calcola_similarita(previsione, estrazione):
    return len(set(previsione) & set(estrazione))

# Funzione per testare diverse combinazioni di pesi
def ottimizza_pesi(previsione_ml, top_frequenti, ritardatari, estrazioni_reali):
    best_weight_model = 0
    best_weight_stat = 0
    best_precision = 0

    # Prova diverse combinazioni di pesi (grid search)
    for weight_model in np.arange(0, 1.1, 0.1):
        for weight_stat in np.arange(0, 1.1, 0.1):
            if weight_model + weight_stat > 1:  # La somma dei pesi non deve superare 1
                continue

            previsione_combinata = []

            # Aggiungi le previsioni del modello ponderate
            for num in previsione_ml:
                if num not in previsione_combinata and len(previsione_combinata) < 6:
                    previsione_combinata.append(num)

            # Aggiungi i numeri frequenti ponderati
            for num in top_frequenti:
                if num not in previsione_combinata and len(previsione_combinata) < 6:
                    previsione_combinata.append(num)

            # Aggiungi i numeri ritardatari ponderati
            for num in ritardatari[:3]:
                if num not in previsione_combinata and len(previsione_combinata) < 6:
                    previsione_combinata.append(num)

            # Se non raggiungi 6 numeri, aggiungi casualmente
            while len(previsione_combinata) < 6:
                random_number = np.random.choice(ritardatari)  # Aggiungi un numero ritardatario casuale
                if random_number not in previsione_combinata:
                    previsione_combinata.append(random_number)

            # Calcola la similarità tra la previsione e le estrazioni reali
            total_similarity = 0
            for estrazione in estrazioni_reali:
                total_similarity += calcola_similarita(previsione_combinata, estrazione)

            # Calcola la precisione media
            precisione = total_similarity / len(estrazioni_reali)

            # Aggiorna la miglior combinazione di pesi
            if precisione > best_precision:
                best_precision = precisione
                best_weight_model = weight_model
                best_weight_stat = weight_stat

    return best_weight_model, best_weight_stat, best_precision


# Test su un sottoinsieme di estrazioni reali (ultime 50 estrazioni)
estrazioni_reali = df.tail(50).values.tolist()  # Usa le ultime 50 estrazioni

# Ottimizzazione dei pesi
best_weight_model, best_weight_stat, best_precision = ottimizza_pesi(
    predictions[-1].astype(int),  # Predizioni del modello
    [num for num, freq in numeri_ordinati_frequenza[:5]],  # Top 5 numeri frequenti
    ritardatari,  # Numeri ritardatari
    estrazioni_reali
)

print(f"\nMigliori pesi trovati: modello={best_weight_model}, statistica={best_weight_stat}")
print(f"Precisione media ottenuta: {best_precision:.2f}")

# Previsione finale con i pesi ottimizzati
previsione_combinata_ottimizzata = []

# Aggiungi le previsioni del modello ponderate
for num in predictions[-1].astype(int):
    if num not in previsione_combinata_ottimizzata and len(previsione_combinata_ottimizzata) < 6:
        previsione_combinata_ottimizzata.append(num)

# Aggiungi i numeri più frequenti ponderati
for num in numeri_ordinati_frequenza[:5]:
    if num[0] not in previsione_combinata_ottimizzata and len(previsione_combinata_ottimizzata) < 6:
        previsione_combinata_ottimizzata.append(num[0])

# Aggiungi i numeri ritardatari ponderati
for num in ritardatari[:3]:
    if num not in previsione_combinata_ottimizzata and len(previsione_combinata_ottimizzata) < 6:
        previsione_combinata_ottimizzata.append(num)

# Se non raggiungi 6 numeri, prendi numeri casuali dai ritardatari
while len(previsione_combinata_ottimizzata) < 6:
    random_number = np.random.choice(ritardatari)  # Aggiungi un numero ritardatario casuale
    if random_number not in previsione_combinata_ottimizzata:
        previsione_combinata_ottimizzata.append(random_number)

# Stampa la previsione finale ottimizzata
print(f"\nPrevisione finale ottimizzata: {previsione_combinata_ottimizzata}")



Migliori pesi trovati: modello=0.0, statistica=0.0
Precisione media ottenuta: 0.57

Previsione finale ottimizzata: [10, 18, 15, 31, 72.0, 40.0]


In [ ]:
# Previsione finale ottimizzata: senza duplicati e con numeri interi
previsione_combinata_ottimizzata = []

# Aggiungi la percentuale di ottimizzazione
percentuale_model = peso_model_ottimizzato * 100  # Pesi ottimizzati per il modello, espressi come percentuali
percentuale_stat = peso_stat_ottimizzato * 100  # Pesi ottimizzati per la statistica, espressi come percentuali

# Prendi l'ultima estrazione
ultima_estrazione = df_cleaned.tail(1).values.flatten()  # Consideriamo l'ultima estrazione, se necessario puoi regolare questo

# Aggiungi le previsioni ponderate (senza duplicati)
previsione_combinata_ottimizzata.extend([int(num) for num in predictions[-1].astype(int)[:int(6 * peso_model_ottimizzato)]])

# Aggiungi i numeri frequenti ponderati (senza duplicati)
previsione_combinata_ottimizzata.extend([num for num, _ in numeri_ordinati_frequenza[:int(6 * peso_stat_ottimizzato)]])

# Aggiungi numeri ritardatari se necessario (senza duplicati)
for num in ritardatari[:3]:
    # Check if num is not NaN before appending
    if not pd.isnull(num) and num not in previsione_combinata_ottimizzata and len(previsione_combinata_ottimizzata) < 6:
        previsione_combinata_ottimizzata.append(num)

# Elimina duplicati dalla previsione finale
previsione_combinata_ottimizzata = list(set(previsione_combinata_ottimizzata))

# Rimuovi i numeri che sono già usciti nell'ultima estrazione
previsione_combinata_ottimizzata = [num for num in previsione_combinata_ottimizzata if num not in ultima_estrazione]

# Se ci sono meno di 6 numeri, aggiungi numeri mancanti dalla lista dei numeri ritardatari
while len(previsione_combinata_ottimizzata) < 6:
    for rit in ritardatari:
        # Check if rit is not NaN before appending
        if not pd.isnull(rit) and rit not in previsione_combinata_ottimizzata:
            previsione_combinata_ottimizzata.append(rit)
        if len(previsione_combinata_ottimizzata) == 6:
            break

# Convertili tutti in interi
previsione_combinata_ottimizzata = [int(num) for num in previsione_combinata_ottimizzata]

# Stampa la previsione finale con la percentuale di ottimizzazione
print(f"\nPrevisione finale ottimizzata:")
print(f"Numeri previsti: {previsione_combinata_ottimizzata}")
print(f"Percentuale ottimizzata per il modello: {percentuale_model:.2f}%")
print(f"Percentuale ottimizzata per la statistica: {percentuale_stat:.2f}%")



Previsione finale ottimizzata:
Numeri previsti: [72, 40, 10, 41, 18, 2]
Percentuale ottimizzata per il modello: 50.00%
Percentuale ottimizzata per la statistica: 50.00%


Previsione e salvataggio in CSV

In [ ]:
import numpy as np
import pandas as pd

# Predizione della prossima estrazione
ultima_sequenza = X[-1].reshape(1, X.shape[1], X.shape[2])
predizione = modello.predict(ultima_sequenza)

# Denormalizza la predizione per riportare i valori tra 1 e 90
predizione_denorm = scaler.inverse_transform(predizione)

# Arrotonda e limita i valori tra 1 e 90
def trasforma_predizione(predizione_denorm):
    predizione_arrotondata = np.round(predizione_denorm).astype(int)  # Arrotonda e converte in int
    predizione_arrotondata = np.clip(predizione_arrotondata, 1, 90)  # Assicura i numeri validi
    return predizione_arrotondata

# Applica la trasformazione
predizione_interi = trasforma_predizione(predizione_denorm)

# Controllo prima di salvare
print(f"Predizione della prossima estrazione: {predizione_interi.flatten()}")

# Creazione del DataFrame con il formato corretto
df_predizione = pd.DataFrame(predizione_interi, columns=["Num1", "Num2", "Num3", "Num4", "Num5"])

# Salvataggio corretto in CSV
csv_path = "/content/prossima_estrazione.csv"
df_predizione.to_csv(csv_path, index=False, header=True)

print(f"File CSV salvato correttamente in {csv_path}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
Predizione della prossima estrazione: [1 1 1 1 1]
File CSV salvato correttamente in /content/prossima_estrazione.csv


<ipython-input-114-0635bc2e6773>:13: RuntimeWarning: invalid value encountered in cast
  predizione_arrotondata = np.round(predizione_denorm).astype(int)  # Arrotonda e converte in int


In [ ]:

import pandas as pd
import datetime
import zipfile

file_path = '/content/it-superenalotto-past-draws-archive.zip'

def carica_database(file_path):
    # Estrai il file .xls dal file .zip
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        xls_file = zip_ref.namelist()[0]  # Ottieni il nome del file .xls all'interno dello zip
        with zip_ref.open(xls_file) as file:
            # Leggi il file Excel usando il file estratto
            df = pd.read_excel(file, engine='xlrd', skiprows=1)
    # Prende solo numeri e rimuove colonne vuote
    numeri_estratti = df.select_dtypes(include='number').dropna(axis=1, how='all')

    # Usa solo le ultime 50 estrazioni 🔥
    numeri_estratti = numeri_estratti.tail(50)
    return numeri_estratti

def calcola_frequenze(df):
    frequenze = df.stack().value_counts()
    return frequenze.head(5).index.tolist()

def calcola_ritardatari(df):
    numeri_totali = set(range(1, 91))
    ultime_uscite = {}

    for numero in numeri_totali:
        try:
            ultima_pos = df.apply(lambda colonna: numero in colonna.values).iloc[::-1].idxmax()
            ultime_uscite[numero] = ultima_pos
        except:
            ultime_uscite[numero] = -1

    ritardatari = sorted(ultime_uscite, key=ultime_uscite.get, reverse=True)[:5]
    return ritardatari

def somma_90(previsione):
    somma = (previsione[0] + previsione[1]) % 90
    if somma == 0:
        somma = 90
    return somma

# Esegui tutto
df = carica_database(file_path)
numeri_frequenti = calcola_frequenze(df)
numeri_ritardatari = calcola_ritardatari(df)
previsione = numeri_frequenti + numeri_ritardatari
somma = somma_90(previsione)

print(f"Data aggiornamento: {datetime.date.today()}")
print("Numeri più frequenti:", numeri_frequenti)
print("Numeri ritardatari:", numeri_ritardatari)
print("Previsione Finale:", previsione)
print("🔥 Somma 90:", somma)



*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
Data aggiornamento: 2025-03-10
Numeri più frequenti: [72.0, 77.0, 40.0, 41.0, 85.0]
Numeri ritardatari: [1, 6, 8, 9, 13]
Previsione Finale: [72.0, 77.0, 40.0, 41.0, 85.0, 1, 6, 8, 9, 13]
🔥 Somma 90: 59.0


Previsione Finale


In [ ]:
import pandas as pd
import numpy as np

# Caricamento delle ultime 10 estrazioni
file_path = '/content/it-superenalotto-past-draws-archive.zip'
df = carica_database(file_path)  # Funzione che abbiamo già definito
ultime_estrazioni = df.tail(200).values  # Prendiamo solo i valori numerici

# Previsioni ottenute
previsione_ml = [26, 36, 57, 70, 90]  # Sostituisci con la tua previsione ML
top_numeri_stat = [72, 41, 40, 77, 83, 1, 6, 8, 9, 13]  # Previsione statistica

# Funzione per contare i match tra previsione e ultime estrazioni
def conta_match(previsione, estrazioni):
    match_per_estrazione = [len(set(previsione) & set(estr)) for estr in estrazioni]
    totale_match = sum(match_per_estrazione)
    return totale_match, match_per_estrazione

# Confronto tra previsioni e ultime estrazioni
match_ml, dettagli_ml = conta_match(previsione_ml, ultime_estrazioni)
match_stat, dettagli_stat = conta_match(top_numeri_stat, ultime_estrazioni)

# Analisi frequenze
frequenze = df.stack().value_counts().head(15)  # Prendi i 15 numeri più frequenti

# Output risultati
print(f"🎯 Confronto con le ultime 10 estrazioni:")
print(f"✔️ Match con ML: {match_ml} numeri trovati")
print(f"✔️ Match con Statistica: {match_stat} numeri trovati")
print("\n🔍 Dettaglio delle corrispondenze per ogni estrazione:")
print(f"ML: {dettagli_ml}")
print(f"Statistica: {dettagli_stat}")
print("\n🔥 Numeri più frequenti nelle ultime estrazioni:")
print(frequenze)

# Suggerimento per la prossima previsione
suggeriti = set(frequenze.index[:5]) | set(top_numeri_stat[:5])  # Unione dei top numeri
print("\n📌 Numeri suggeriti per la prossima estrazione:")
print(suggeriti)



*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
🎯 Confronto con le ultime 10 estrazioni:
✔️ Match con ML: 7 numeri trovati
✔️ Match con Statistica: 35 numeri trovati

🔍 Dettaglio delle corrispondenze per ogni estrazione:
ML: [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Statistica: [0, 0, 1, 0, 0, 0, 1, 3, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 3, 3, 0, 1, 1, 2, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]

🔥 Numeri più frequenti nelle ultime estrazioni:
72.0    9
77.0    6
40.0    6
41.0    6
85.0    5
63.0    5
83.0    5
30.0    5
29.0    5
59.0    4
87.0    4
25.0    4
49.0    4
74.0    4
86.0    4
Name: count, dtype: int64

📌 Numeri suggeriti per la prossima estrazione:
{72.0, 41.0, 40.0, 77.0, 83, 85.0}


In [ ]:
import numpy as np

# Funzione migliorata per preparare i dati (controllo dei NaN)
def prepara_dati(df_cleaned, sequenza_lenght=5):
    sequenze = []
    target = []

    for i in range(len(df_cleaned) - sequenza_lenght):
        # Estrazione dei numeri per la sequenza
        sequenza = df_cleaned.iloc[i:i + sequenza_lenght].values.flatten()

        # Se la sequenza contiene NaN, la scartiamo
        if np.any(np.isnan(sequenza)):
            continue

        sequenze.append(sequenza)
        target.append(df_cleaned.iloc[i + sequenza_lenght].values.flatten())

    X = np.array(sequenze)
    y = np.array(target)

    return X, y

# Prepara i dati
X, y = prepara_dati(df_cleaned)

# Verifica la forma dei dati
print(f"Forma di X: {X.shape}")
print(f"Forma di y: {y.shape}")


Forma di X: (33, 25)
Forma di y: (33, 5)


Terza Cella: Prepara i dati per l'allenamento

In [ ]:
import numpy as np

# Funzione per preparare i dati: crea sequenze temporali
def prepara_dati(df_cleaned, sequenza_lenght=5):
    sequenze = []
    target = []

    # Cicliamo per creare le sequenze
    for i in range(len(df_cleaned) - sequenza_lenght):
        sequenza = df_cleaned.iloc[i:i + sequenza_lenght].values.flatten()
        sequenze.append(sequenza)
        target.append(df_cleaned.iloc[i + sequenza_lenght].values.flatten())

    # Convertiamo in array numpy
    X = np.array(sequenze)
    y = np.array(target)

    return X, y

# Prepara i dati per l'allenamento
X, y = prepara_dati(df_cleaned)

# Mostra la forma di X e y per verificarne la corretta preparazione
print(f"Forma di X: {X.shape}")
print(f"Forma di y: {y.shape}")


Forma di X: (35, 25)
Forma di y: (35, 5)


Quarta Cella: Creazione e allenamento del modello

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Creazione del modello di rete neurale
model = Sequential()
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))  # Primo strato
model.add(Dense(64, activation='relu'))  # Secondo strato
model.add(Dense(y.shape[1], activation='linear'))  # Strato di uscita per predizioni numeriche

# Compilazione del modello con una funzione di perdita appropriata per la regressione
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['accuracy'])

# Allenamento del modello
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Salviamo il modello allenato per usi futuri
model.save("/content/lotto_model.h5")



Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_los

In [ ]:
# Se X è 2D, aggiungiamo una dimensione per le caratteristiche
if len(X.shape) == 2:
    X = np.expand_dims(X, axis=-1)  # Aggiungiamo una dimensione per le features

print(X.shape)  # Ora X dovrebbe essere (n_samples, sequence_length, 1)

# Creazione del modello LSTM
modello = Sequential()
modello.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
modello.add(LSTM(units=50))
modello.add(Dense(units=5))  # Predizione di 5 numeri per ogni sequenza

# Compilazione del modello
modello.compile(optimizer=Adam(), loss='mean_squared_error')

# Addestramento del modello
modello.fit(X, y, epochs=10, batch_size=32)

# Predizione della prossima estrazione
predizione = modello.predict(X[-1].reshape(1, X.shape[1], X.shape[2]))  # Utilizza l'ultima sequenza per la previsione

# Funzione per trasformare la previsione in numeri interi tra 1 e 90
def trasforma_predizione(predizione):
    predizione_arrotondata = np.round(predizione)  # Arrotonda i valori continui
    predizione_arrotondata = np.clip(predizione_arrotondata, 1, 90)  # Limita i valori tra 1 e 90
    return predizione_arrotondata.astype(int)  # Converte in interi

# Trasforma la predizione in numeri interi da 1 a 90
predizione_interi = trasforma_predizione(predizione)
# Se X è 2D, aggiungiamo una dimensione per le caratteristiche
if len(X.shape) == 2:
    X = np.expand_dims(X, axis=-1)  # Aggiungiamo una dimensione per le features

print(X.shape)  # Ora X dovrebbe essere (n_samples, sequence_length, 1)

# Creazione del modello LSTM
modello = Sequential()
modello.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
modello.add(LSTM(units=50))
modello.add(Dense(units=5))  # Predizione di 5 numeri per ogni sequenza

# Compilazione del modello
modello.compile(optimizer=Adam(), loss='mean_squared_error')

# Addestramento del modello
modello.fit(X, y, epochs=10, batch_size=32)

# Predizione della prossima estrazione
predizione = modello.predict(X[-1].reshape(1, X.shape[1], X.shape[2]))  # Utilizza l'ultima sequenza per la previsione

# Funzione per trasformare la previsione in numeri interi tra 1 e 90
def trasforma_predizione(predizione):
    predizione_arrotondata = np.round(predizione)  # Arrotonda i valori continui
    predizione_arrotondata = np.clip(predizione_arrotondata, 1, 90)  # Limita i valori tra 1 e 90
    return predizione_arrotondata.astype(int)  # Converte in interi

# Trasforma la predizione in numeri interi da 1 a 90
predizione_interi = trasforma_predizione(predizione)

print(f"Predizione della prossima estrazione: {predizione_interi}")

(35, 25, 1)
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: nan
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: nan
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: nan
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan 
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: nan 
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: nan 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
(35, 25, 1)
Epoch 1/10


<ipython-input-51-eb2057967d88>:26: RuntimeWarning: invalid value encountered in cast
  return predizione_arrotondata.astype(int)  # Converte in interi


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: nan
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: nan 
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan 
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan 
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: nan
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
Predizione della prossima estrazione: [[-9223372036854775808 -9223372036854775808 -9223372036854775808
  -9223372036854775808 -9223372036854775808]]


<ipython-input-51-eb2057967d88>:55: RuntimeWarning: invalid value encountered in cast
  return predizione_arrotondata.astype(int)  # Converte in interi


Quinta Cella: Fare previsioni

noooo

next step

In [ ]:
# Genera una previsione (esempio semplificato)
# La previsione si basa sui numeri più frequenti
previsione = numeri_frequenti

NameError: name 'numeri_frequenti' is not defined

In [ ]:
# Crea un DataFrame per la previsione
previsione_df = pd.DataFrame({'Numero': previsione})


In [ ]:
# Salva la previsione in un file CSV
previsione_df.to_csv('/content/previsione_lotto.csv', index=False)

print("Previsione salvata in /content/previsione_lotto.csv")

from google.colab import files
files.download('/content/previsione_lotto.csv')


Previsione salvata in /content/previsione_lotto.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>